# Gesture Recognition

>> - Sanjaya Kumar Sahu
>> - Adarsh Vittal Shetty


- In this group project, we are building 2 models that will be able to predict the 5 gestures correctly.
>>- Model 1: CNN+RNN Model
>>- Model 2: Conv3d Model

In [8]:
#Import all the packages
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import cv2
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv2D,Conv3D, MaxPooling3D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras


We set the random seed so that the results don't vary drastically.

In [9]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [10]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10 # Increase the batch size and check the performance of GPU
n_frame = 30 # number of frames
y = 120 # image width
z = 120 # image height
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop 
channel = 3
curr_dt_time = datetime.datetime.now()
source_path=train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


# training sequences = 663
# validation sequences = 100
# epochs = 10


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [11]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,n_frame)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,n_frame,y,z,3)) # n_frame is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    tempImage = cv2.resize(image, (y,z), interpolation = cv2.INTER_AREA)              
                    
                    batch_data[folder,idx,:,:,0] = (tempImage[:,:,0])/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (tempImage[:,:,1])/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (tempImage[:,:,2])/255.0 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            #print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,n_frame,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    tempImage = cv2.resize(image, (y,z), interpolation = cv2.INTER_AREA)          
                    
                    batch_data[folder,idx,:,:,0] = (tempImage[:,:,0])/255.0
                    batch_data[folder,idx,:,:,1] = (tempImage[:,:,1])/255.0
                    batch_data[folder,idx,:,:,2] = (tempImage[:,:,2])/255.0
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels         


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model 1
- CNN+RNN Architecture
- We are using GRU architecture for RNN layer

In [12]:
#write your model here
nb_featuremap = [8,16,32,64]
nb_classes = 5

# Input shape
input_shape=(n_frame,y,z,3)

model = Sequential()

model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))


model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add((Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add((Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(nb_classes, activation='softmax')) # using Softmax as last layer

In [13]:
 #write your optimizer
learning_rate=0.001
optimiser = optimizers.Adam(learning_rate) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 60, 60, 16)    64        
_________________________________________________________________
activation_4 (Activation)    (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_16 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 30, 30, 30, 32)    128       
__________

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

#### Experiment 1 : Batch Size - 10, Epoch - 10

In [15]:
batch_size = 10 # Increase the batch size and check the performance of GPU
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 10
<generator object generator at 0x7f86280774c0>
<generator object generator at 0x7f85fce33ba0>
Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 180s 3s/step - loss: 1.2885 - categorical_accuracy: 0.4711 - val_loss: 1.9702 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-1.27740-0.46908-1.97016-0.41000.h5
Epoch 2/10
67/67 [==============================] - 14s 207ms/step - loss: 1.3344 - categorical_accuracy: 0.4925 - val_loss: 2.8949 - val_categorical_accuracy: 0.2800

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-1.33443-0.49254-2.89488-0.28000.h5
Epoch 3/10
67/67 [==============================] - 17s 258ms/step - loss: 1.4474 - categorical_accuracy: 0.4030 - val_loss: 2.9803 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030

#### Experiment 2 : Batch Size - 20, Epoch - 10

In [16]:
batch_size = 20 # Increase the batch size and check the performance of GPU
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 10
<generator object generator at 0x7f85fce33fc0>
<generator object generator at 0x7f86280774c0>
Source path =  Project_data/val ; batch size = 20
Source path =  Project_data/train Epoch 1/10
; batch size = 20
34/34 [==============================] - 49s 1s/step - loss: 0.6589 - categorical_accuracy: 0.7645 - val_loss: 0.9380 - val_categorical_accuracy: 0.6300

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.61927-0.77526-0.93800-0.63000.h5
Epoch 2/10
34/34 [==============================] - 8s 228ms/step - loss: 0.9675 - categorical_accuracy: 0.5882 - val_loss: 0.9432 - val_categorical_accuracy: 0.6700

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.96748-0.58824-0.94316-0.67000.h5
Epoch 3/10
34/34 [==============================] - 8s 249ms/step - loss: 1.1006 - categorical_accuracy: 0.5882 - val_loss: 1.0175 - val_categorical_accuracy: 0.6400

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328

#### Experiment 3 : Batch Size - 30, Epoch - 10

In [17]:
batch_size = 30 # Increase the batch size and check the performance of GPU
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 10
<generator object generator at 0x7f862a22ff10>
<generator object generator at 0x7f85e06432b0>
Source path =  Project_data/val ; batch size = 30
Source path =  Epoch 1/10
Project_data/train ; batch size = 30
23/23 [==============================] - 49s 2s/step - loss: 0.6234 - categorical_accuracy: 0.7854 - val_loss: 1.2321 - val_categorical_accuracy: 0.6200

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.53104-0.81599-1.23213-0.62000.h5
Epoch 2/10
23/23 [==============================] - 4s 176ms/step - loss: 0.8221 - categorical_accuracy: 0.6377 - val_loss: 1.1090 - val_categorical_accuracy: 0.6500

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.82207-0.63768-1.10904-0.65000.h5
Epoch 3/10
23/23 [==============================] - 6s 241ms/step - loss: 0.9510 - categorical_accuracy: 0.5942 - val_loss: 0.9009 - val_categorical_accuracy: 0.6250

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328

#### Experiment 4 : Batch Size - 30, Epoch - 20

In [18]:
batch_size = 30 # Increase the batch size and check the performance of GPU
num_epochs = 20 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 20
<generator object generator at 0x7f862a1cd570>
<generator object generator at 0x7f862a22ff10>
Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/trainEpoch 1/20
 ; batch size = 30
23/23 [==============================] - 49s 2s/step - loss: 0.6420 - categorical_accuracy: 0.7883 - val_loss: 1.0935 - val_categorical_accuracy: 0.6200

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.53542-0.81900-1.09346-0.62000.h5
Epoch 2/20
23/23 [==============================] - 5s 203ms/step - loss: 0.9197 - categorical_accuracy: 0.6522 - val_loss: 0.7512 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.91970-0.65217-0.75117-0.75000.h5
Epoch 3/20
23/23 [==============================] - 6s 247ms/step - loss: 0.7868 - categorical_accuracy: 0.7246 - val_loss: 1.2064 - val_categorical_accuracy: 0.6000

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328

#### Experiment 5 : Batch Size - 30, Epoch - 30

In [19]:
batch_size = 30 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f85fce33fc0>
<generator object generator at 0x7f862a1cd570>
Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/30
23/23 [==============================] - 49s 2s/step - loss: 0.6485 - categorical_accuracy: 0.7921 - val_loss: 1.2405 - val_categorical_accuracy: 0.6000

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.55293-0.80995-1.24049-0.60000.h5
Epoch 2/30
23/23 [==============================] - 5s 223ms/step - loss: 0.8350 - categorical_accuracy: 0.6667 - val_loss: 0.9777 - val_categorical_accuracy: 0.6000

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.83498-0.66667-0.97768-0.60000.h5
Epoch 3/30
23/23 [==============================] - 6s 240ms/step - loss: 0.8327 - categorical_accuracy: 0.7536 - val_loss: 0.9402 - val_categorical_accuracy: 0.7250

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328

#### Experiment 6 : Batch Size - 20, Epoch - 20

In [20]:
batch_size = 20 # Increase the batch size and check the performance of GPU
num_epochs = 20 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 20
<generator object generator at 0x7f85e06bbaf0>
<generator object generator at 0x7f85e06bb2b0>
Source path =  Project_data/val ; batch size = 20
Source path =  Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 49s 1s/step - loss: 0.6056 - categorical_accuracy: 0.7800 - val_loss: 0.8962 - val_categorical_accuracy: 0.6900

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.56375-0.79940-0.89622-0.69000.h5
Epoch 2/20
34/34 [==============================] - 8s 234ms/step - loss: 0.7937 - categorical_accuracy: 0.6275 - val_loss: 0.8974 - val_categorical_accuracy: 0.6700

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.79367-0.62745-0.89745-0.67000.h5
Epoch 3/20
34/34 [==============================] - 8s 245ms/step - loss: 0.8355 - categorical_accuracy: 0.6667 - val_loss: 0.8944 - val_categorical_accuracy: 0.6700

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328

#### Experiment 7 : Batch Size - 10, Epoch - 20

In [21]:
batch_size = 10 # Increase the batch size and check the performance of GPU
num_epochs = 20 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 20
<generator object generator at 0x7f85e06bb678>
<generator object generator at 0x7f85e06bbaf0>
Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 50s 741ms/step - loss: 0.6078 - categorical_accuracy: 0.7946 - val_loss: 0.8992 - val_categorical_accuracy: 0.6700

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.59717-0.79940-0.89919-0.67000.h5
Epoch 2/20
67/67 [==============================] - 16s 242ms/step - loss: 0.9054 - categorical_accuracy: 0.6517 - val_loss: 0.9148 - val_categorical_accuracy: 0.6600

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.90539-0.65174-0.91484-0.66000.h5
Epoch 3/20
67/67 [==============================] - 15s 231ms/step - loss: 0.9099 - categorical_accuracy: 0.6517 - val_loss: 0.9029 - val_categorical_accuracy: 0.6700

Epoch 00003: saving model to model_init_2020-06-2912_27_34.0

#### Experiment 8 : Batch Size - 10, Epoch - 30

In [22]:
batch_size = 10 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f85e091e0a0>
<generator object generator at 0x7f85fcb77360>
Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - 49s 737ms/step - loss: 0.6316 - categorical_accuracy: 0.7777 - val_loss: 0.9020 - val_categorical_accuracy: 0.6800

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.60596-0.78582-0.90205-0.68000.h5
Epoch 2/30
67/67 [==============================] - 15s 223ms/step - loss: 0.9469 - categorical_accuracy: 0.6716 - val_loss: 0.9056 - val_categorical_accuracy: 0.6600

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.94687-0.67164-0.90564-0.66000.h5
Epoch 3/30
67/67 [==============================] - 16s 239ms/step - loss: 0.8285 - categorical_accuracy: 0.6716 - val_loss: 0.9173 - val_categorical_accuracy: 0.6700

Epoch 00003: saving model to model_init_2020-06-2912_27_34.0

#### Experiment 9 : Batch Size - 40, Epoch - 30

In [23]:
batch_size = 40 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f85fce33e60>
<generator object generator at 0x7f85fca149e8>
Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/30
17/17 [==============================] - 51s 3s/step - loss: 0.7118 - categorical_accuracy: 0.7796 - val_loss: 1.2686 - val_categorical_accuracy: 0.6000

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.64324-0.79638-1.26859-0.60000.h5
Epoch 2/30
17/17 [==============================] - 28s 2s/step - loss: 0.5474 - categorical_accuracy: 0.7877 - val_loss: 0.9720 - val_categorical_accuracy: 0.7000

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.54740-0.78772-0.97201-0.70000.h5
Epoch 3/30
17/17 [==============================] - 26s 2s/step - loss: 0.6275 - categorical_accuracy: 0.7928 - val_loss: 0.6971 - val_categorical_accuracy: 0.7167

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/mod

17/17 [==============================] - 21s 1s/step - loss: 0.6164 - categorical_accuracy: 0.7855 - val_loss: 0.8619 - val_categorical_accuracy: 0.6667

Epoch 00027: saving model to model_init_2020-06-2912_27_34.030328/model-00027-0.61638-0.78547-0.86192-0.66667.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 3.388131949944826e-24.
Epoch 28/30
17/17 [==============================] - 21s 1s/step - loss: 0.5234 - categorical_accuracy: 0.8097 - val_loss: 0.9149 - val_categorical_accuracy: 0.7000

Epoch 00028: saving model to model_init_2020-06-2912_27_34.030328/model-00028-0.52336-0.80969-0.91488-0.70000.h5
Epoch 29/30
17/17 [==============================] - 21s 1s/step - loss: 0.5635 - categorical_accuracy: 0.7958 - val_loss: 0.9390 - val_categorical_accuracy: 0.6333

Epoch 00029: saving model to model_init_2020-06-2912_27_34.030328/model-00029-0.56347-0.79585-0.93900-0.63333.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.694065974972413e-24.
Epoch 30/30
1

#### Experiment 10 : Batch Size - 20, Epoch - 30

In [24]:
batch_size = 20 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f85fce33ca8>
<generator object generator at 0x7f862a1cd570>
Source path =  Project_data/val ; batch size = 20
Source path =  Project_data/train ; batch size = 20
Epoch 1/30
34/34 [==============================] - 49s 1s/step - loss: 0.6604 - categorical_accuracy: 0.7829 - val_loss: 0.8970 - val_categorical_accuracy: 0.6800

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.57948-0.80241-0.89700-0.68000.h5
Epoch 2/30
34/34 [==============================] - 9s 260ms/step - loss: 0.9961 - categorical_accuracy: 0.6667 - val_loss: 0.8943 - val_categorical_accuracy: 0.6900

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.99607-0.66667-0.89434-0.69000.h5
Epoch 3/30
34/34 [==============================] - 9s 266ms/step - loss: 1.1522 - categorical_accuracy: 0.5588 - val_loss: 0.9045 - val_categorical_accuracy: 0.6700

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328

#### Experiment 11 : Batch Size - 60, Epoch - 30

In [25]:
batch_size = 60 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f85fccd6db0>
<generator object generator at 0x7f85fc710ba0>
Source path =  Project_data/val ; batch size =Source path =  Project_data/train ; batch size = 60
 60
Epoch 1/30
12/12 [==============================] - 50s 4s/step - loss: 0.5850 - categorical_accuracy: 0.7939 - val_loss: 1.9064 - val_categorical_accuracy: 0.4600

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.52533-0.80392-1.90640-0.46000.h5
Epoch 2/30
12/12 [==============================] - 3s 236ms/step - loss: 0.6930 - categorical_accuracy: 0.8056 - val_loss: 0.8932 - val_categorical_accuracy: 0.6625

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.69305-0.80556-0.89320-0.66250.h5
Epoch 3/30
12/12 [==============================] - 3s 226ms/step - loss: 0.7826 - categorical_accuracy: 0.6667 - val_loss: 1.7265 - val_categorical_accuracy: 0.4750

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328

12/12 [==============================] - 4s 311ms/step - loss: 0.8338 - categorical_accuracy: 0.6667 - val_loss: 0.9428 - val_categorical_accuracy: 0.7000

Epoch 00027: saving model to model_init_2020-06-2912_27_34.030328/model-00027-0.83377-0.66667-0.94280-0.70000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 2.0194840132861292e-31.
Epoch 28/30
12/12 [==============================] - 4s 325ms/step - loss: 0.8867 - categorical_accuracy: 0.6111 - val_loss: 0.9267 - val_categorical_accuracy: 0.7000

Epoch 00028: saving model to model_init_2020-06-2912_27_34.030328/model-00028-0.88670-0.61111-0.92670-0.70000.h5
Epoch 29/30
12/12 [==============================] - 4s 305ms/step - loss: 1.0553 - categorical_accuracy: 0.6667 - val_loss: 0.7460 - val_categorical_accuracy: 0.6500

Epoch 00029: saving model to model_init_2020-06-2912_27_34.030328/model-00029-1.05531-0.66667-0.74601-0.65000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.0097420066430646e-31.
Epoch

# Gesture Recognition using CONV3D:

## Model 2
- CONV3D model


In [26]:
#write your model here
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))
model_a.add(MaxPooling3D(pool_size=(2,2,2)))


model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))
model_a.add(MaxPooling3D(pool_size=(2,2,2)))


model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))
model_a.add(MaxPooling3D(pool_size=(2,2,2)))


model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))
model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())
model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [27]:
 #write your optimizer
learning_rate=0.001
optimiser = optimizers.Adam(learning_rate) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_7 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_8 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [28]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator at 0x7f85e06bbdb0>
<generator object generator at 0x7f85fccd6db0>


In [29]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [30]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

#### Experiment 0 : Batch Size - 60, Epoch - 30

In [31]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 60
Source path =  Project_data/train ; batch size = 60
Epoch 1/30
12/12 [==============================] - 54s 5s/step - loss: 4.8319 - categorical_accuracy: 0.2293 - val_loss: 3.5883 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-4.94218-0.24736-3.58827-0.21000.h5
Epoch 2/30
12/12 [==============================] - 3s 223ms/step - loss: 4.0439 - categorical_accuracy: 0.2222 - val_loss: 1.9314 - val_categorical_accuracy: 0.4875

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-4.04392-0.22222-1.93136-0.48750.h5
Epoch 3/30
12/12 [==============================] - 2s 178ms/step - loss: 3.9265 - categorical_accuracy: 0.2500 - val_loss: 4.0510 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-3.92652-0.25000-4.05102-0.40000.h5
Epoch 4/30
12/12 [==============================] - 3s 220ms


Epoch 00027: saving model to model_init_2020-06-2912_27_34.030328/model-00027-1.43808-0.33333-1.16809-0.52500.h5
Epoch 28/30
12/12 [==============================] - 4s 310ms/step - loss: 1.3849 - categorical_accuracy: 0.3889 - val_loss: 1.1986 - val_categorical_accuracy: 0.5750

Epoch 00028: saving model to model_init_2020-06-2912_27_34.030328/model-00028-1.38493-0.38889-1.19858-0.57500.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
Epoch 29/30
12/12 [==============================] - 4s 301ms/step - loss: 1.6226 - categorical_accuracy: 0.2778 - val_loss: 1.2122 - val_categorical_accuracy: 0.6000

Epoch 00029: saving model to model_init_2020-06-2912_27_34.030328/model-00029-1.62263-0.27778-1.21221-0.60000.h5
Epoch 30/30
12/12 [==============================] - 4s 305ms/step - loss: 1.7014 - categorical_accuracy: 0.2500 - val_loss: 1.0500 - val_categorical_accuracy: 0.6000

Epoch 00030: saving model to model_init_2020-06-2912_27_34.030328/model-00

In [32]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
# Define model b
model_b = Sequential()
model_b.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=(n_frame,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Conv3D(128, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))
model_b.add(Flatten())

model_b.add(Dense(128,activation='relu'))
model_b.add(BatchNormalization())
model_b.add(Dropout(0.5))

model_b.add(Dense(64,activation='relu'))
model_b.add(BatchNormalization())
model_b.add(Dropout(0.25))

model_b.add(Dense(classes, activation='softmax'))



model_b.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_b.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_11 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  13856     
_________________________________________________________________
activation_12 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

#### Experiment 1 : Batch Size - 20, Epoch - 10

In [33]:
batch_size = 20 # Increase the batch size and check the performance of GPU
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs

In [34]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [35]:

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [36]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size =Source path =  Project_data/train Epoch 1/10
 20
; batch size = 20
34/34 [==============================] - 80s 2s/step - loss: 2.0665 - categorical_accuracy: 0.2546 - val_loss: 5.3163 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-2.05327-0.26094-5.31634-0.23000.h5
Epoch 2/10
34/34 [==============================] - 15s 434ms/step - loss: 1.7402 - categorical_accuracy: 0.2549 - val_loss: 3.7437 - val_categorical_accuracy: 0.1900

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-1.74023-0.25490-3.74374-0.19000.h5
Epoch 3/10
34/34 [==============================] - 15s 433ms/step - loss: 1.6562 - categorical_accuracy: 0.3529 - val_loss: 12.0797 - val_categorical_accuracy: 0.1800

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-1.65619-0.35294-12.07974-0.18000.h5
Epoch 4/10
34/34 [==============================] - 15s 

#### Experiment 2 : Batch Size - 30, Epoch - 10

In [37]:
batch_size=30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [38]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [39]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = Source path =  Project_data/train Epoch 1/10
; batch size = 3030

23/23 [==============================] - 90s 4s/step - loss: 1.6941 - categorical_accuracy: 0.3310 - val_loss: 2.6917 - val_categorical_accuracy: 0.1500

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-1.64793-0.34389-2.69170-0.15000.h5
Epoch 2/10
23/23 [==============================] - 9s 402ms/step - loss: 1.7438 - categorical_accuracy: 0.2464 - val_loss: 2.2922 - val_categorical_accuracy: 0.1750

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-1.74377-0.24638-2.29218-0.17500.h5
Epoch 3/10
23/23 [==============================] - 9s 402ms/step - loss: 1.6519 - categorical_accuracy: 0.3333 - val_loss: 2.1562 - val_categorical_accuracy: 0.1750

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-1.65192-0.33333-2.15617-0.17500.h5
Epoch 4/10
23/23 [==============================] - 9s 398ms

In [40]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

nb_frames=30
nb_rows=120
nb_cols=120
nb_channel =3

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model_c = Sequential()

model_c.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_c.add(Activation('relu'))
model_c.add(Dropout(0.25))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_c.add(Flatten())

model_c.add(Dense(nb_dense[0], activation='relu'))
model_c.add(Dropout(0.5))

model_c.add(Dense(nb_dense[1], activation='relu'))
model_c.add(Dropout(0.5))

#softmax layer
model_c.add(Dense(nb_dense[2], activation='softmax'))


learning_rate=0.001
optimiser = optimizers.Adam(learning_rate) #write your optimizer
model_c.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_c.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_17 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_15 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_18 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_16 (Activation)   (None, 15, 60, 60, 16)    0         
__________

#### Experiment 3 : Batch Size - 40, Epoch - 10

In [41]:
batch_size=40
print ('# epochs =', num_epochs)
print ('# batch_size =', batch_size)

# epochs = 10
# batch_size = 40


In [42]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [43]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [44]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/10
17/17 [==============================] - 54s 3s/step - loss: 2.3315 - categorical_accuracy: 0.2039 - val_loss: 1.5895 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-2.34843-0.20362-1.58951-0.20000.h5
Epoch 2/10
17/17 [==============================] - 26s 2s/step - loss: 1.5895 - categorical_accuracy: 0.2532 - val_loss: 1.5195 - val_categorical_accuracy: 0.2833

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-1.58954-0.25320-1.51948-0.28333.h5
Epoch 3/10
17/17 [==============================] - 27s 2s/step - loss: 1.5555 - categorical_accuracy: 0.2709 - val_loss: 1.5001 - val_categorical_accuracy: 0.4333

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-1.55611-0.26703-1.50006-0.43333.h5
Epoch 4/10
17/17 [==============================] - 24s 1s/step 

#### Experiment 4 : Batch Size - 40, Epoch - 15

In [45]:
batch_size = 40
num_epochs = 15
print ('# epochs =', num_epochs)
print ('# batch_size =', batch_size)

# epochs = 15
# batch_size = 40


In [46]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [47]:

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator at 0x7f86280774c0>
<generator object generator at 0x7f85d807deb8>


In [48]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = Epoch 1/15
40
17/17 [==============================] - 51s 3s/step - loss: 1.1522 - categorical_accuracy: 0.5475 - val_loss: 1.4759 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-1.10917-0.55656-1.47592-0.32000.h5
Epoch 2/15
17/17 [==============================] - 29s 2s/step - loss: 1.0461 - categorical_accuracy: 0.5806 - val_loss: 1.1173 - val_categorical_accuracy: 0.5500

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-1.04610-0.58056-1.11730-0.55000.h5
Epoch 3/15
17/17 [==============================] - 27s 2s/step - loss: 0.9977 - categorical_accuracy: 0.6285 - val_loss: 1.2601 - val_categorical_accuracy: 0.3667

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-0.98375-0.62943-1.26006-0.36667.h5
Epoch 4/15
17/17 [==============================] - 24s 1s/step 

#### Experiment 5 : Batch Size - 40, Epoch - 20

In [49]:
batch_size=40
num_epochs = 20 

In [50]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [51]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator at 0x7f85fca149e8>
<generator object generator at 0x7f86280774c0>


In [52]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40Epoch 1/20

17/17 [==============================] - 52s 3s/step - loss: 0.8185 - categorical_accuracy: 0.7611 - val_loss: 1.0591 - val_categorical_accuracy: 0.5900

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.72292-0.77526-1.05909-0.59000.h5
Epoch 2/20
17/17 [==============================] - 29s 2s/step - loss: 0.6111 - categorical_accuracy: 0.7545 - val_loss: 0.8669 - val_categorical_accuracy: 0.6500

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.61114-0.75448-0.86693-0.65000.h5
Epoch 3/20
17/17 [==============================] - 27s 2s/step - loss: 0.7031 - categorical_accuracy: 0.7640 - val_loss: 0.7519 - val_categorical_accuracy: 0.6833

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-0.68822-0.76839-0.75186-0.68333.h5
Epoch 4/20
17/17 [==============================] - 24s 1s/step 

#### Experiment 6 : Batch Size - 40, Epoch - 30

In [53]:
batch_size=40
num_epochs = 30 
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator at 0x7f85c07f8a40>
<generator object generator at 0x7f85fca149e8>


In [54]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size =Epoch 1/30
 40
17/17 [==============================] - 52s 3s/step - loss: 0.6826 - categorical_accuracy: 0.7724 - val_loss: 1.3076 - val_categorical_accuracy: 0.5700

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.62387-0.78582-1.30760-0.57000.h5
Epoch 2/30
17/17 [==============================] - 29s 2s/step - loss: 0.5649 - categorical_accuracy: 0.7801 - val_loss: 0.7844 - val_categorical_accuracy: 0.7000

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.56486-0.78005-0.78436-0.70000.h5
Epoch 3/30
17/17 [==============================] - 28s 2s/step - loss: 0.6491 - categorical_accuracy: 0.7743 - val_loss: 0.7704 - val_categorical_accuracy: 0.6667

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-0.63922-0.77657-0.77045-0.66667.h5
Epoch 4/30
17/17 [==============================] - 24s 1s/step 

#### Experiment 7 : Batch Size - 30, Epoch - 10

In [55]:
batch_size=30
num_epochs = 10 

In [56]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [57]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator at 0x7f85c07f5308>
<generator object generator at 0x7f85c07f8a40>


In [58]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val Source path =  Project_data/train ; batch size = 30
; batch size = 30
Epoch 1/10
23/23 [==============================] - 52s 2s/step - loss: 0.5451 - categorical_accuracy: 0.8171 - val_loss: 1.1175 - val_categorical_accuracy: 0.6500

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.54947-0.80995-1.11749-0.65000.h5
Epoch 2/10
23/23 [==============================] - 3s 150ms/step - loss: 0.7543 - categorical_accuracy: 0.6667 - val_loss: 0.9056 - val_categorical_accuracy: 0.5750

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.75427-0.66667-0.90556-0.57500.h5
Epoch 3/10
23/23 [==============================] - 5s 228ms/step - loss: 0.6233 - categorical_accuracy: 0.8406 - val_loss: 0.6780 - val_categorical_accuracy: 0.7750

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-0.62329-0.84058-0.67797-0.77500.h5
Epoch 4/10
23/23 [==============================] - 5s 225ms

#### Experiment 8 : Batch Size - 10, Epoch - 40

In [59]:
batch_size = 10
num_epochs = 40

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size =Epoch 1/40 10

23/23 [==============================] - 18s 774ms/step - loss: 0.6387 - categorical_accuracy: 0.7696 - val_loss: 0.8668 - val_categorical_accuracy: 0.5750

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.63873-0.76957-0.86675-0.57500.h5
Epoch 2/40
23/23 [==============================] - 17s 755ms/step - loss: 0.5497 - categorical_accuracy: 0.7870 - val_loss: 0.7594 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.54972-0.78696-0.75939-0.75000.h5
Epoch 3/40
23/23 [==============================] - 16s 699ms/step - loss: 0.7298 - categorical_accuracy: 0.7908 - val_loss: 0.6165 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-0.59317-0.81340-0.61647-0.80000.h5
Epoch 4/40
23/23 [==============================] - 6s 

23/23 [==============================] - 5s 208ms/step - loss: 0.7328 - categorical_accuracy: 0.7246 - val_loss: 0.7667 - val_categorical_accuracy: 0.7500

Epoch 00027: saving model to model_init_2020-06-2912_27_34.030328/model-00027-0.73280-0.72464-0.76671-0.75000.h5
Epoch 28/40
23/23 [==============================] - 5s 214ms/step - loss: 0.8236 - categorical_accuracy: 0.6667 - val_loss: 0.9360 - val_categorical_accuracy: 0.6000

Epoch 00028: saving model to model_init_2020-06-2912_27_34.030328/model-00028-0.82363-0.66667-0.93597-0.60000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 7.275957959772868e-15.
Epoch 29/40
23/23 [==============================] - 6s 244ms/step - loss: 0.8402 - categorical_accuracy: 0.7101 - val_loss: 0.6467 - val_categorical_accuracy: 0.7250

Epoch 00029: saving model to model_init_2020-06-2912_27_34.030328/model-00029-0.84016-0.71014-0.64674-0.72500.h5
Epoch 30/40
23/23 [==============================] - 5s 236ms/step - loss: 0.8251 - cate

#### Experiment 9 : Batch Size - 10, Epoch - 25

In [60]:
batch_size = 10
num_epochs = 25

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/25
23/23 [==============================] - 18s 800ms/step - loss: 0.5940 - categorical_accuracy: 0.7957 - val_loss: 0.8440 - val_categorical_accuracy: 0.6000

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.59397-0.79565-0.84401-0.60000.h5
Epoch 2/25
23/23 [==============================] - 17s 726ms/step - loss: 0.5725 - categorical_accuracy: 0.7826 - val_loss: 0.5363 - val_categorical_accuracy: 0.8000

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.57248-0.78261-0.53628-0.80000.h5
Epoch 3/25
23/23 [==============================] - 16s 702ms/step - loss: 0.6270 - categorical_accuracy: 0.7666 - val_loss: 0.9470 - val_categorical_accuracy: 0.6000

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-0.63296-0.76555-0.94701-0.60000.h5
Epoch 4/25
23/23 [==============================] - 6s 

#### Experiment 10 : Batch Size - 20, Epoch - 25

In [61]:
batch_size = 20
num_epochs = 25

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 20
Source path =  Project_data/train ; batch size = 20
Epoch 1/25
23/23 [==============================] - 38s 2s/step - loss: 0.5468 - categorical_accuracy: 0.8022 - val_loss: 0.7905 - val_categorical_accuracy: 0.7000

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.54679-0.80217-0.79051-0.70000.h5
Epoch 2/25
23/23 [==============================] - 17s 759ms/step - loss: 0.8137 - categorical_accuracy: 0.7015 - val_loss: 0.7162 - val_categorical_accuracy: 0.7000

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.64563-0.75314-0.71616-0.70000.h5
Epoch 3/25
23/23 [==============================] - 6s 252ms/step - loss: 0.6396 - categorical_accuracy: 0.7826 - val_loss: 0.8973 - val_categorical_accuracy: 0.6250

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-0.63956-0.78261-0.89731-0.62500.h5
Epoch 4/25
23/23 [==============================] - 6s 253m

#### Experiment 11 : Batch Size - 5, Epoch - 30

In [62]:
batch_size = 5
num_epochs = 30

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 5
Source path =  Project_data/train ; batch size =Epoch 1/30
 5
23/23 [==============================] - 10s 418ms/step - loss: 0.5761 - categorical_accuracy: 0.7652 - val_loss: 0.8156 - val_categorical_accuracy: 0.6500

Epoch 00001: saving model to model_init_2020-06-2912_27_34.030328/model-00001-0.57607-0.76522-0.81564-0.65000.h5
Epoch 2/30
23/23 [==============================] - 9s 392ms/step - loss: 0.5797 - categorical_accuracy: 0.8000 - val_loss: 0.8230 - val_categorical_accuracy: 0.7000

Epoch 00002: saving model to model_init_2020-06-2912_27_34.030328/model-00002-0.57973-0.80000-0.82300-0.70000.h5
Epoch 3/30
23/23 [==============================] - 9s 376ms/step - loss: 0.4498 - categorical_accuracy: 0.8696 - val_loss: 0.6372 - val_categorical_accuracy: 0.7500

Epoch 00003: saving model to model_init_2020-06-2912_27_34.030328/model-00003-0.44984-0.86957-0.63718-0.75000.h5
Epoch 4/30
23/23 [==============================] - 8s 367m

23/23 [==============================] - 5s 207ms/step - loss: 0.7551 - categorical_accuracy: 0.7246 - val_loss: 0.8526 - val_categorical_accuracy: 0.7000

Epoch 00027: saving model to model_init_2020-06-2912_27_34.030328/model-00027-0.75507-0.72464-0.85263-0.70000.h5
Epoch 28/30
23/23 [==============================] - 5s 208ms/step - loss: 0.9735 - categorical_accuracy: 0.6522 - val_loss: 0.9926 - val_categorical_accuracy: 0.4500

Epoch 00028: saving model to model_init_2020-06-2912_27_34.030328/model-00028-0.97350-0.65217-0.99257-0.45000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 2.6469780858943953e-26.
Epoch 29/30
23/23 [==============================] - 5s 229ms/step - loss: 0.5206 - categorical_accuracy: 0.8551 - val_loss: 0.7339 - val_categorical_accuracy: 0.7500

Epoch 00029: saving model to model_init_2020-06-2912_27_34.030328/model-00029-0.52057-0.85507-0.73390-0.75000.h5
Epoch 30/30
23/23 [==============================] - 5s 212ms/step - loss: 0.7979 - cat

# Observation On Different Experiments:

## Model 1 : CNN+RNN Trainable params: 1,902,005

### Experiment:1 on Batch Size =10, Epoch = 10
Epoch 10/10
loss: 0.9069 - categorical_accuracy: 0.6119 - val_loss: 0.9850 - val_categorical_accuracy: 0.6300

### Experiment:2 on Batch Size =20, Epoch = 10
Epoch 10/10
loss: 0.9174 - categorical_accuracy: 0.7353 - val_loss: 0.9396 - val_categorical_accuracy: 0.6700

### Experiment:3 on Batch Size =30, Epoch = 10
Epoch 10/10
loss: 0.7290 - categorical_accuracy: 0.7391 - val_loss: 1.0953 - val_categorical_accuracy: 0.5750

### Experiment:4 on Batch Size =30, Epoch = 20
Epoch 20/20
loss: 0.7787 - categorical_accuracy: 0.6522 - val_loss: 1.1214 - val_categorical_accuracy: 0.6500

### Experiment:5 on Batch Size =30, Epoch = 30
Epoch 30/30
loss: 0.9082 - categorical_accuracy: 0.6377 - val_loss: 1.0167 - val_categorical_accuracy: 0.6500

### Experiment:6 on Batch Size =20, Epoch = 20
Epoch 20/20
loss: 0.9323 - categorical_accuracy: 0.5882 - val_loss: 0.9121 - val_categorical_accuracy: 0.6600

### Experiment:7 on Batch Size =10, Epoch = 20
Epoch 20/20
loss: 0.8509 - categorical_accuracy: 0.6816 - val_loss: 0.9085 - val_categorical_accuracy: 0.6700

### Experiment:8 on Batch Size =10, Epoch = 30
Epoch 30/30
loss: 0.8549 - categorical_accuracy: 0.6667 - val_loss: 0.9035 - val_categorical_accuracy: 0.6800


### Experiment:9 on Batch Size =40, Epoch = 30
Epoch 7/30
loss: 0.5894 - categorical_accuracy: 0.8131 - val_loss: 0.6884 - val_categorical_accuracy: 0.7667


### Experiment:10 on Batch Size =20, Epoch = 30
Epoch 30/30
loss: 0.9784 - categorical_accuracy: 0.6569 - val_loss: 0.9091 - val_categorical_accuracy: 0.6700

### Experiment:11 on Batch Size =60, Epoch = 30
Epoch 30/30
loss: 0.7104 - categorical_accuracy: 0.6944 - val_loss: 1.0876 - val_categorical_accuracy: 0.6500





## Model 2 : CONV3D

## Model_b Trainable params: 1,120,133

### Experiment:1 on Batch Size =20, Epoch = 10
Epoch 10/10
loss: 1.7627 - categorical_accuracy: 0.3431 - val_loss: 2.4863 - val_categorical_accuracy: 0.1600

### Experiment:2 on Batch Size =30, Epoch = 10
Epoch 10/10
loss: 1.7355 - categorical_accuracy: 0.3188 - val_loss: 1.6884 - val_categorical_accuracy: 0.3750


## Model_c Trainable params: 863,989

### Experiment:3 on Batch Size =40, Epoch = 10
Epoch 10/10
loss: 1.1334 - categorical_accuracy: 0.5329 - val_loss: 1.2390 - val_categorical_accuracy: 0.4333

### Experiment:4 on Batch Size =40, Epoch = 15
Epoch 15/15
loss: 0.6567 - categorical_accuracy: 0.7578 - val_loss: 0.8244 - val_categorical_accuracy: 0.7500

### Experiment:5 on Batch Size =40, Epoch = 20
Epoch 20/20
loss: 0.5407 - categorical_accuracy: 0.8062 - val_loss: 0.7622 - val_categorical_accuracy: 0.7167

### Experiment:6 on Batch Size =40, Epoch = 30
Epoch 30/30
loss: 0.5679 - categorical_accuracy: 0.7647 - val_loss: 0.7589 - val_categorical_accuracy: 0.6833

### Experiment:7 on Batch Size =30, Epoch = 10
Epoch 10/10
loss: 0.6244 - categorical_accuracy: 0.7971 - val_loss: 0.6635 - val_categorical_accuracy: 0.7250

### Experiment:8 on Batch Size =10, Epoch = 40
Epoch 40/40
loss: 0.7909 - categorical_accuracy: 0.6522 - val_loss: 0.7220 - val_categorical_accuracy: 0.7000

### Experiment:9 on Batch Size =10, Epoch = 25
Epoch 25/25
loss: 0.7981 - categorical_accuracy: 0.6667 - val_loss: 0.8166 - val_categorical_accuracy: 0.6000


### Experiment:10 on Batch Size =20, Epoch = 25
Epoch 25/25
loss: 0.7638 - categorical_accuracy: 0.6812 - val_loss: 0.8124 - val_categorical_accuracy: 0.6750


### Experiment:11 on Batch Size =5, Epoch = 30
Epoch 30/30
loss: 0.7979 - categorical_accuracy: 0.6957 - val_loss: 0.7131 - val_categorical_accuracy: 0.8500


